In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
data_beverage = pd.read_csv('/content/drive/MyDrive/ITMO/Анализ данных Python/ЛР №5 (разбиение выборки) /beverage_r.csv', delimiter=';', index_col='numb.obs')

In [3]:
data_beverage.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34 entries, 1 to 34
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   COKE     34 non-null     int64
 1   D_COKE   34 non-null     int64
 2   D_PEPSI  34 non-null     int64
 3   D_7UP    34 non-null     int64
 4   PEPSI    34 non-null     int64
 5   SPRITE   34 non-null     int64
 6   TAB      34 non-null     int64
 7   SEVENUP  34 non-null     int64
dtypes: int64(8)
memory usage: 2.4 KB


**Задача №1**

Разделить таблицу данных так, чтобы в обучающую выборку попало 66% наблюдений, а в тестовую выборку 33% наблюдений

In [4]:
data_beverage_train, data_beverage_test = train_test_split(data_beverage, train_size=0.66, random_state=42)

In [5]:
data_beverage_train.info()
data_beverage_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22 entries, 17 to 29
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   COKE     22 non-null     int64
 1   D_COKE   22 non-null     int64
 2   D_PEPSI  22 non-null     int64
 3   D_7UP    22 non-null     int64
 4   PEPSI    22 non-null     int64
 5   SPRITE   22 non-null     int64
 6   TAB      22 non-null     int64
 7   SEVENUP  22 non-null     int64
dtypes: int64(8)
memory usage: 1.5 KB
<class 'pandas.core.frame.DataFrame'>
Index: 12 entries, 16 to 5
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   COKE     12 non-null     int64
 1   D_COKE   12 non-null     int64
 2   D_PEPSI  12 non-null     int64
 3   D_7UP    12 non-null     int64
 4   PEPSI    12 non-null     int64
 5   SPRITE   12 non-null     int64
 6   TAB      12 non-null     int64
 7   SEVENUP  12 non-null     int64
dtypes: int64(8)
memory usage: 864.0 bytes


**Задача №2**

Разделить таблицу данных на четыре одинаковых части.

In [6]:
from sklearn.model_selection import KFold

In [7]:
X = np.array(data_beverage)
kf = KFold(n_splits=4, shuffle=True, random_state=42)
parts = []
for _, test_index in kf.split(X):
    X_part = X[test_index]
    parts.append((X_part))


In [8]:
data_beverage.shape, len(parts), len(parts[0])
# Проверка. Сходится

((34, 8), 4, 9)

**Задача №3**

Разделить таблицу данных на две стратифицированные подвыборки.
По переменным кока-кола и минералка.

In [9]:
data_beverage.head()

,COKE,D_COKE,D_PEPSI,D_7UP,PEPSI,SPRITE,TAB,SEVENUP
numb.obs,,,,,,,,
1,1,0,0,0,1,1,0,1
2,1,0,0,0,1,0,0,0
3,1,0,0,0,1,0,0,0
4,0,1,0,1,0,0,1,0
5,1,0,0,0,1,0,0,0


In [36]:
# Объявление стратификационных переменных
var1 = 'COKE'
var2 = 'TAB'

# Распределение переменных в выборке
data_beverage[var1].mean(), data_beverage[var2].mean()

(np.float64(0.5882352941176471), np.float64(0.3235294117647059))

In [11]:
# Создание составной стратификационную переменную
data_beverage['stratify_composite'] = data_beverage[var1].astype(str) + '_' + data_beverage[var2].astype(str)

# Уникальные комбинации для стратификации и распределение
data_beverage['stratify_composite'].nunique()
data_beverage['stratify_composite'].value_counts()

,count
stratify_composite,
1_0,19
0_1,10
0_0,4
1_1,1


In [14]:
# Проверка, репрезантативной ли будет стратификация
min_count = data_beverage['stratify_composite'].value_counts().min()
min_count

1

In [20]:
data_beverage[data_beverage['stratify_composite'] == '1_1'].index.to_list()

[9]

In [21]:
if min_count < 2:
    data_beverage_inplace = data_beverage.drop(9)
data_beverage_inplace['stratify_composite'].value_counts()

,count
stratify_composite,
1_0,19
0_1,10
0_0,4


In [23]:
# Разделение по стратам (получение индексов)
train_idx, test_idx = train_test_split(
    data_beverage_inplace.index,
    test_size=0.5,
    random_state=42,
    stratify=data_beverage_inplace['stratify_composite'])

In [26]:
# Создание стратификационных выборок
train_beverage = data_beverage_inplace.loc[train_idx].copy()
test_beverage = data_beverage_inplace.loc[test_idx].copy()

In [33]:
# Проверка корректности стратификации
print(f"Страты тренировочной выборки: {len(train_beverage)} ({len(train_beverage)/len(data_beverage_inplace)*100:.1f}%)")
print(f"Страты тестовой выборки: {len(test_beverage)} ({len(test_beverage)/len(data_beverage_inplace)*100:.1f}%)")

Страты тренировочной выборки: 16 (48.5%)
Страты тестовой выборки: 17 (51.5%)


In [30]:
train_beverage['stratify_composite'].value_counts()

,count
stratify_composite,
1_0,9
0_1,5
0_0,2


In [48]:
check_strat = list(map(float, [data_beverage[var1].mean(),
            data_beverage[var2].mean(),
            train_beverage[var1].mean(),
            train_beverage[var2].mean()]))
check_strat

[0.5882352941176471, 0.3235294117647059, 0.5625, 0.3125]